In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from bs4 import BeautifulSoup
from selenium.common.exceptions import TimeoutException, NoSuchElementException, StaleElementReferenceException
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
import re
import pandas as pd
from pytube import Search
import os
from tqdm import tqdm
import time
import traceback
import json
from lxml import html
import requests
from bs4 import BeautifulSoup
from urllib.parse import quote
from concurrent.futures import ThreadPoolExecutor, as_completed
from selenium.webdriver.common.proxy import Proxy, ProxyType
from fake_useragent import UserAgent
import random
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from selenium.webdriver.chrome.options import Options
from selenium.webdriver import ChromeOptions
import pickle

In [2]:
def url_append_song(seq, max_page, proxy=None):
    div_list_song = []
    max_elements = 50
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36'
    }

    for page_number in range(1, max_page + 1):
        start_index = (page_number - 1) * max_elements + 1
        ajax_url = f"https://www.melon.com/dj/playlist/djplaylist_listsong.htm?startIndex={start_index}&pageSize=50&plylstSeq={seq}"
        try:
            response = requests.get(ajax_url, headers=headers, proxies=proxy)
            if response.status_code == 200:
                soup = BeautifulSoup(response.content, 'html.parser')
                for i in range(1, max_elements + 1):
                    selector = f'tr:nth-child({i}) > td:nth-child(4) > div > a'
                    element = soup.select_one(selector)
                    if element and element.has_attr('href'):
                        js_code = element['href']
                        match = re.search(r'(\d+)\D*$', js_code)
                        if match:
                            div_list_song.append(match.group(1))
                    else:
                        break
            else:
                print(f"Failed to fetch AJAX URL: HTTP {response.status_code}")
                break  # 중요: HTTP 상태 코드가 200이 아닐 경우 루프를 중단합니다.

        except Exception as e:
            print(f"Error during requests.get: {e} - URL: {ajax_url}")
            break  # 예외 발생 시 루프를 중단합니다.

    return div_list_song




def save_dictionary_to_json(data, file_path):
    with open(file_path, 'w', encoding='utf-8') as file:
        json.dump(data, file, ensure_ascii=False, indent=4)
def red_text(text):
    return f"\033[91m{text}\033[0m"

In [3]:


def extract_lyric(soup):
    lyrics_div = soup.find('div', {'id': 'd_video_summary'})
    if lyrics_div:
        lyrics = lyrics_div.get_text(separator='\n').strip()
        lyrics = lyrics.replace('\n', ' ').strip()  # 줄바꿈을 공백으로 치환
        lyrics = re.sub(r'\s+', ' ', lyrics)        # 연속된 공백을 하나의 공백으로 치환
        return lyrics
    else:
        return "아직 가사가 등록되지 않았습니다."



import re

def extract_song_name(soup):
    try:
        song_name_element = soup.select_one('div.entry > div.info > div.song_name')
        if song_name_element:
            song_name = song_name_element.get_text()
            # 줄바꿈과 탭을 공백으로 치환
            song_name = song_name.replace('\n', ' ').replace('\t', ' ')
            # 연속된 공백을 한 개의 공백으로 치환
            clean_song_name = re.sub(r'\s+', ' ', song_name).strip()
            clean_song_name = clean_song_name[3:]
            return clean_song_name
        else:
            return "Unknown"
    except Exception as e:
        print(f"Error extracting song name: {e}")
        return "Unknown"



def extract_other_data(soup):
    extracted_data = {}

    # 아티스트 추출
    artist_element = soup.select_one("div.info > div.artist > a > span:nth-child(1)")
    extracted_data['artist'] = artist_element.get_text().strip() if artist_element else "Unknown"

    # 발매일 추출
    release_date_element = soup.select_one("div.meta > dl > dd:nth-child(4)")
    extracted_data['release_date'] = release_date_element.get_text().strip() if release_date_element else "Unknown"

    # 앨범 추출
    album_element = soup.select_one("div.meta > dl > dd:nth-child(2) > a")
    extracted_data['album'] = album_element.get_text().strip() if album_element else "Unknown"

    # 장르 추출
    genre_element = soup.select_one("div.meta > dl > dd:nth-child(6)")
    extracted_data['genre'] = genre_element.get_text().strip() if genre_element else "Unknown"

    return extracted_data


def extract_texts(url, proxy=None):
    data_dict = {
        'song_name': "Unknown",
        'artist': "Unknown",
        'release_date': "Unknown",
        'album': "Unknown",
        'lyric': "아직 가사가 등록되지 않았습니다.",
        'genre': "Unknown"
    }

    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36'
    }

    try:
        response = requests.get(url, headers=headers, proxies=proxy)
        if response.status_code == 200:
            soup = BeautifulSoup(response.content, 'html.parser')

            # 곡 제목 추출
            data_dict['song_name'] = extract_song_name(soup)

            # 기타 데이터 (아티스트, 발매일, 앨범, 장르) 추출
            other_data = extract_other_data(soup)
            data_dict.update(other_data)

            # 가사 추출
            data_dict['lyric'] = extract_lyric(soup)
        else:
            print(f"Failed to retrieve webpage: HTTP {response.status_code}")

    except Exception as e:
        print(f"An error occurred in extract_texts for URL: {url} - {e}")

    return data_dict




In [4]:
selectors = {
    'artist': "div.info > div.artist > a > span:nth-child(1)",
    'release_date': "div.meta > dl > dd:nth-child(4)",
    'album': "div.meta > dl > dd:nth-child(2) > a",
    'genre': "div.meta > dl > dd:nth-child(6)"  # 장르에 대한 선택자를 추가
}
def get_song_info(div_list, proxy):
    combined_dict = {
        'song_name': [],
        'artist': [],
        'release_date': [],
        'album': [],
        'genre': [],
        'lyric': []
    }

    for div in div_list:
        song_info = extract_texts(url=f'https://www.melon.com/song/detail.htm?songId={div}', proxy = proxy)

        for key in combined_dict:
            combined_dict[key].append(song_info[key])

    return combined_dict

In [5]:
def create_playlist_dictionary(playlist_seq, proxy=None):
    playlist_url = f'https://www.melon.com/mymusic/dj/mymusicdjplaylistview_inform.htm?plylstSeq={playlist_seq}'
    print(f'Extracting on {playlist_seq}' )
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.105 Safari/537.36',
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
        'Accept-Language': 'en-US,en;q=0.9',
        'Accept-Encoding': 'gzip, deflate, br',
    }
    try:
        response = requests.get(playlist_url, headers=headers, proxies=proxy, timeout=10)
        if response.status_code == 200:
            soup = BeautifulSoup(response.content, 'html.parser')
            playlist_name_element = soup.select_one('#conts > div.section_info.d_djcol_list > div > div.entry > div.info > div.ellipsis.song_name')
            playlist_name = playlist_name_element.text if playlist_name_element else 'Unknown'
            playlist_name = playlist_name.replace('\n', '').replace('\t', '').strip()
            
            total_elements_element = soup.select_one('#conts > div.section_contin > div.page_header > h5 > span.sum')
            total_elements_text = total_elements_element.text if total_elements_element else '0'
            total_elements = int(re.search(r'\((\d+)\)', total_elements_text).group(1)) if '(' in total_elements_text else 0
            max_page = (total_elements + 49) // 50
            
            song_info = get_song_info(url_append_song(playlist_seq, max_page, proxy), proxy)
            if song_info is None or not song_info:  # song_info가 None이거나 비어있는 경우
                print(f"Failed to get songs for playlist {playlist_seq}")
                return {'playlist_name': 'Unknown', 'song_info': []}
            return {
                'playlist_name': playlist_name,
                'song_info': song_info
            }
        else:
            print(f"Failed to retrieve webpage: HTTP {response.status_code}")
            return {'playlist_name': 'Unknown', 'song_info': []}
    except Exception as e:
        print(f"An error occurred in create_playlist_dictionary: {e}")
        return {'playlist_name': 'Unknown', 'song_info': []}



In [6]:
def parse_proxy_string(proxy_string):
    proxy_list = []
    for line in proxy_string.splitlines():
        if line.strip():
            ip, port = line.split(':')
            proxy_list.append(f"http://{ip}:{port}")
    return proxy_list

In [7]:

def process_playlist(div, index, path_to_save, proxy):
    playlist_dict = create_playlist_dictionary(div)
    json_filename = f'playlist{index}.json'
    json_filepath = os.path.join(path_to_save, json_filename)
    save_dictionary_to_json(playlist_dict, json_filepath)


# 저장할 경로 설정
path_to_save = 'C:\\Temp\\unge'

# 각 스레드에서 실행될 함수
def process_with_proxy(proxy, href, index, path_to_save):
    try:
        process_playlist(href, index, path_to_save, proxy)
    except Exception as e:
        print(f"An error occurred in process_with_proxy: {e}")


def process_playlist_safe(href, index, path_to_save):
    try:
        proxy_index = index % len(combined_proxy_list)
        proxy = combined_proxy_list[proxy_index]
        proxy_config = {'http': proxy, 'https': proxy}
        process_with_proxy(proxy_config, href, index, path_to_save)
    except Exception as e:
        print(f"An error occurred in process_playlist_safe: {e}")


In [ ]:
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm
err = []
file_path = 'C:\\Temp\\unge\\proxy.txt'
with open(file_path, 'r', encoding='utf-8') as file:
    proxy_string =  file.read()

combined_proxy_list = parse_proxy_string(proxy_string)
path_to_save = 'C:\\Temp\\unge'

# 프록시 리스트와 hrefs 리스트를 사용합니다.
# 예를 들어, proxy_list = [<프록시 목록>] 와 같이 정의되어 있어야 합니다.

file_path = 'C:\\Temp\\unge\\real_list.pickle'

# 파일에서 리스트 불러오기
with open(file_path, 'rb') as file:
    hrefs = pickle.load(file)

print(len(hrefs))
# ThreadPoolExecutor를 사용하여 작업을 병렬로 처리
with ThreadPoolExecutor(max_workers=8) as executor:
    tasks = []
    for i, href in enumerate(hrefs):
        # 올바른 인자 순서로 함수 호출
        tasks.append(executor.submit(process_playlist_safe, href, i, path_to_save))

    for task in tqdm(as_completed(tasks), total=len(tasks), desc="Processing playlists"):
        task.result()



165438
Extracting on 496712391
Extracting on 481919862
Extracting on 423155202
Extracting on 101640392
Extracting on 489878864
Extracting on 473724566
Extracting on 400725248
Extracting on 437468365


Processing playlists:   0%|                                                     | 1/165438 [00:07<325:50:49,  7.09s/it]

Extracting on 101617392


Processing playlists:   0%|                                                     | 2/165438 [00:12<289:40:04,  6.30s/it]

Extracting on 437356973


...

Extracting on 517537857


Processing playlists:   0%|                                                    | 47/165438 [04:56<587:48:21, 12.79s/it]

Extracting on 100120970


Processing playlists:   0%|                                                    | 48/165438 [05:12<643:31:46, 14.01s/it]

Extracting on 526850401


#### 

['496712391',
 '481919862',
 '423155202',
 '101640392',
 '489878864',
 '473724566',
 '400725248',
 '437468365',
 '101617392',
 '437356973',
 '407314240',
 '403942447',
 '408851622',
 '528379679',
 '477556837',
 '442590860',
 '466580421',
 '485581388',
 '441687608',
 '101565978',
 '514450511',
 '523676548',
 '478320922',
 '527539787',
 '480480119',
 '494896790',
 '524572752',
 '421541621',
 '469001044',
 '505856648',
 '415481770',
 '522269245',
 '521109571',
 '485528321',
 '100076388',
 '460610894',
 '521038085',
 '409248442',
 '420033547',
 '455015016',
 '436509753',
 '507632186',
 '509822357',
 '438697118',
 '420552168',
 '498284024',
 '443404056',
 '422840170',
 '518329861',
 '435085497',
 '438017851',
 '449148761',
 '404033142',
 '517537857',
 '100120970',
 '526850401',
 '101609117',
 '408416817',
 '527755979',
 '512348419',
 '469708720',
 '517702595',
 '481910089',
 '101656894',
 '100124891',
 '528883946',
 '441029211',
 '452777878',
 '474230156',
 '478186317',
 '520370419',
 '4529

In [ ]:
import requests


def check_proxy(proxy):
    try:
        proxies = {
            "http": f"http://{proxy['ip']}:{proxy['port']}",
            "https": f"http://{proxy['ip']}:{proxy['port']}"
        }
        response = requests.get("http://httpbin.org/ip", proxies=proxies, timeout=5)
        response.raise_for_status()
        return True
    except:
        return False

for proxy in combined_proxy_list:
    if check_proxy(proxy):
        print(f"Proxy {proxy['ip']}:{proxy['port']} works!")
    else:
        print(f"Proxy {proxy['ip']}:{proxy['port']} failed!")
